## Table of Content
* [Make RBC Model](#chapter1)
    * [Model Equations](#section_1_1)
    * [Find Steady State, Solve the Model](#section_1_2)
* [Get Italy data](#chapter2)

In [1]:
%matplotlib inline
import gEcon as ge
import gEcon.plotting as gp
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd

import arviz as az

import sys
sys.path.append('..')
from TimeSeriesTools.stats import ADF_test_summary
from TimeSeriesTools.transformers import DetrendTransformer, LogTransformer

config = {
    'figure.figsize':(14,4),
    'figure.dpi':144,
    'figure.facecolor':'white',
    'axes.grid':True,
    'grid.linestyle':'--',
    'grid.linewidth':0.5,
    'axes.spines.top':False,
    'axes.spines.bottom':False,
    'axes.spines.left':False,
    'axes.spines.right':False
}

plt.rcParams.update(config)

WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


ModuleNotFoundError: No module named 'TimeSeriesTools'

## Make RBC Model <a class="anchor" id="chapter1"></a>

In [ ]:
rbc = ge.gEconModel('GCN Files/RBC_complete.gcn')

### Model Equations  <a class="anchor" id="section_1_1"></a>

In [ ]:
for equation in rbc.system_equations:
    display(equation)

### Find Steady State, Solve the Model <a class="anchor" id="section_1_2"></a>

In [ ]:
rbc.steady_state()
rbc.solve_model()

In [ ]:
rbc.print_steady_state()

## Get Italy data <a class="anchor" id="chapter2"></a>